In [ ]:
# %%
"""
Stage D: NeRF vs GT geometry evaluation
Sequence: c1_descending_t2_v2

依赖前置：
- Stage A 已经生成：
  - undistorted/rgb/*.png
  - camera_pinhole.json
  - transforms.json
- Nerfstudio 已经训练完一个 run：
  - 有对应的 config.yml（ns-train 输出目录）
- C3VDv2 raw 目录下有 coverage_mesh.obj
"""

import os
import json
import math
from pathlib import Path

import numpy as np
import torch
import open3d as o3d
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

device = "cuda"
print("Using device:", device)

# %% [markdown]
# ----
# ## D0. 配置路径（***请根据你自己的目录检查一下***）

# %%
SEQ = "c1_descending_t2_v2"

PROJECT_ROOT = Path("/data1_ycao/chua/projects/cdTeacher")
RAW_ROOT     = PROJECT_ROOT / "data_raw" / SEQ
STAGE_A_ROOT = PROJECT_ROOT / "outputs" / "stage_A" / SEQ

# Nerfstudio run 的 config.yml 路径（请改成你真实的路径）
# 例子：/data1_ycao/chua/projects/cdTeacher/outputs/nerfstudio_runs/c3vdv2_multi_seq_c1_descending_t2_v2/nerfacto/2025-11-18_12-34-56/config.yml
NS_CONFIG_PATH = Path(
    "/data1_ycao/chua/projects/cdTeacher/outputs/stage_A/c1_descending_t2_v2/outputs/nerfacto/2025-11-16_042345/config.yml"
)

assert RAW_ROOT.exists(), f"RAW_ROOT not found: {RAW_ROOT}"
assert STAGE_A_ROOT.exists(), f"STAGE_A_ROOT not found: {STAGE_A_ROOT}"
assert NS_CONFIG_PATH.exists(), f"NS_CONFIG_PATH not found: {NS_CONFIG_PATH}"

print("RAW_ROOT    :", RAW_ROOT)
print("STAGE_A_ROOT:", STAGE_A_ROOT)
print("NS_CONFIG   :", NS_CONFIG_PATH)


Using device: cuda:4
RAW_ROOT    : /data1_ycao/chua/projects/cdTeacher/data_raw/c1_descending_t2_v2
STAGE_A_ROOT: /data1_ycao/chua/projects/cdTeacher/outputs/stage_A/c1_descending_t2_v2
NS_CONFIG   : /data1_ycao/chua/projects/cdTeacher/outputs/stage_A/c1_descending_t2_v2/outputs/nerfacto/2025-11-16_042345/config.yml


In [4]:

# %% [markdown]
# ----
# ## D1. 读取 pinhole 相机和 transforms.json

# %%
# camera_pinhole.json
with open(STAGE_A_ROOT / "undistorted" / "camera_pinhole.json", "r") as f:
    cam_pinhole = json.load(f)

W_p = cam_pinhole["width"]
H_p = cam_pinhole["height"]
fx  = cam_pinhole["fx"]
fy  = cam_pinhole["fy"]
cx  = cam_pinhole["cx"]
cy  = cam_pinhole["cy"]

print("Pinhole intrinsics:", cam_pinhole)

# transforms.json
with open(STAGE_A_ROOT / "transforms.json", "r") as f:
    tf_meta = json.load(f)

frames = tf_meta["frames"]
num_frames = len(frames)
print("Num frames in transforms:", num_frames)

# 取出所有 T_cam2world
poses_c2w = np.stack([np.array(fr["transform_matrix"], dtype=np.float32) for fr in frames], axis=0)
poses_c2w = torch.from_numpy(poses_c2w).to(device)  # (N,4,4)
print("poses_c2w:", poses_c2w.shape)


Pinhole intrinsics: {'width': 960, 'height': 720, 'fx': 480.00000000000006, 'fy': 480.00000000000006, 'cx': 480.0, 'cy': 360.0, 'fov_deg': 90.0}
Num frames in transforms: 617
poses_c2w: torch.Size([617, 4, 4])


In [10]:

# %% [markdown]
# ----
# ## D2. 用 Nerfstudio 渲染 depth（世界坐标下的表面）

# 这里我们用 ns-render CLI，让 Nerfstudio 输出 per-frame depth 图像。

# %%
NS_DEPTH_ROOT = "/data1_ycao/chua/projects/cdTeacher/outputs/stage_D/c1_descending_t2_v2/ns_depth"
# NS_DEPTH_ROOT.mkdir(exist_ok=True, parents=True)

print("Depth output dir:", NS_DEPTH_ROOT)


# 运行 ns-render 生成 depth 图像。
# 注意：
# - 需要在 nerfstudio 环境中运行此 cell（或者在 shell 里先手动跑一遍 ns-render）。
# - rendered_image_type 里至少包含 depth。
# - 这里假设 ns-render 能自动读取 NerfstudioData 格式的 transforms.json。


# CONFIG=/data1_ycao/chua/projects/cdTeacher/outputs/stage_A/c1_descending_t2_v2/outputs/nerfacto/2025-11-16_042345/config.yml
# OUT_DIR=/data1_ycao/chua/projects/cdTeacher/outputs/stage_D/c1_descending_t2_v2/ns_depth
# mkdir -p "$OUT_DIR"
# ns-render dataset \
#   --load-config "$CONFIG" \
#   --output-path "$OUT_DIR" \
#   --image-format png \
#   --rendered-output-names depth




Depth output dir: /data1_ycao/chua/projects/cdTeacher/outputs/stage_D/c1_descending_t2_v2/ns_depth


In [24]:

# %% [markdown]
# 跑完之后，NS_DEPTH_ROOT 中应该有若干 `depth/****.png` 或 `.exr`。
# 我们假设保存为 16-bit PNG 格式的深度（单位：米）。
# 如果格式不同，请根据实际情况做转换。

# %%
# 检查一下 depth 目录
depth_dir = Path(NS_DEPTH_ROOT + "/test/depth")
print("depth_dir:", depth_dir, "exists:", depth_dir.exists())
if depth_dir.exists():
    some = sorted(depth_dir.glob("*.png"))[:5]
    print("Sample depth files:", [p.name for p in some])


depth_dir: /data1_ycao/chua/projects/cdTeacher/outputs/stage_D/c1_descending_t2_v2/ns_depth/test/depth exists: True
Sample depth files: ['0010.png', '0020.png', '0030.png', '0040.png', '0050.png']


In [ ]:

# %% [markdown]
# ----
# ## D3. 把 NeRF depth 反投影成世界坐标点云

# %%
def load_nerf_depth(frame_id: int):
    """读取 NeRF 渲染出的 depth 图，假设命名为 0000.png, 0001.png, ..."""
    fname = f"{frame_id:04d}.png"
    d_path = depth_dir / fname
    if not d_path.exists():
        return None

    # 假设是 16-bit PNG，存的是 depth_m * scale
    depth_raw = np.array(Image.open(d_path))
    if depth_raw.dtype == np.uint16:
        # 假设最大值映射到 10m（你也可以根据 ns-render 文档调整）
        depth_m = depth_raw.astype(np.float32) / 65535.0 * 10.0
    else:
        depth_m = depth_raw.astype(np.float32)
    return depth_m  # (H, W)


# 预生成一个 pinhole 像素网格
u = torch.arange(W_p, device=device).view(1, -1).expand(H_p, W_p)
v = torch.arange(H_p, device=device).view(-1, 1).expand(H_p, W_p)

x_cam = (u - cx) / fx
y_cam = (v - cy) / fy
ones  = torch.ones_like(x_cam)
dirs_cam_pinhole = torch.stack([x_cam, y_cam, ones], dim=-1)  # (H_p, W_p, 3)
dirs_cam_pinhole = dirs_cam_pinhole / torch.linalg.norm(dirs_cam_pinhole, dim=-1, keepdim=True)

print("dirs_cam_pinhole:", dirs_cam_pinhole.shape)


# %%
def nerf_points_world_from_depth(frame_id: int, max_points: int = 20000):
    """
    从第 frame_id 帧的 NeRF depth 图生成世界坐标点云。
    """
    depth_m = load_nerf_depth(frame_id)
    if depth_m is None:
        return None

    depth_m = torch.from_numpy(depth_m).to(device=device, dtype=torch.float32)  # (H_p, W_p)

    # 有效像素：深度>0
    valid = depth_m > 0
    if valid.sum() == 0:
        return None

    d_valid = depth_m[valid]                             # (Nv,)
    dirs_valid = dirs_cam_pinhole[valid]                 # (Nv,3)

    print(f"DEBUG: dirs_valid shape: {dirs_valid.shape}")
    print(f"DEBUG: d_valid shape before: {d_valid.shape}")
    print(f"DEBUG: d_valid unsqueezed: {d_valid.unsqueeze(-1).shape}")

    pts_cam = dirs_valid * d_valid.unsqueeze(-1)         # (Nv,3)

    # cam2world pose
    T_c2w = poses_c2w[frame_id]                          # (4,4)
    R = T_c2w[:3, :3]
    t = T_c2w[:3, 3]

    pts_world = (R @ pts_cam.T + t.view(3, 1)).T         # (Nv,3)

    # 随机下采样
    if max_points is not None and pts_world.shape[0] > max_points:
        idx = torch.randperm(pts_world.shape[0], device=device)[:max_points]
        pts_world = pts_world[idx]

    return pts_world


# %%
# 聚合若干帧的 NeRF 点云（可以先只用前 N 帧测试）
nerf_points_list = []
max_frames = num_frames  # 或先 100
for fid in tqdm(range(max_frames), desc="Collect NeRF points"):
    pts_w = nerf_points_world_from_depth(fid, max_points=10000)
    if pts_w is None:
        continue
    nerf_points_list.append(pts_w.cpu())

if len(nerf_points_list) == 0:
    raise RuntimeError("No NeRF points collected, please check depth rendering.")
nerf_points = torch.cat(nerf_points_list, dim=0).numpy().astype(np.float32)
print("NeRF points:", nerf_points.shape)

# 保存成 PLY
nerf_ply_path = STAGE_A_ROOT / "nerf_pointcloud_depth.ply"
pcd_nerf = o3d.geometry.PointCloud()
pcd_nerf.points = o3d.utility.Vector3dVector(nerf_points)
o3d.io.write_point_cloud(str(nerf_ply_path), pcd_nerf)
print("Saved NeRF point cloud to:", nerf_ply_path)


dirs_cam_pinhole: torch.Size([720, 960, 3])


Collect NeRF points:   0%|          | 0/617 [00:00<?, ?it/s]

Collect NeRF points:   2%|▏         | 10/617 [00:00<00:09, 66.30it/s]

DEBUG: dirs_valid shape: torch.Size([2073600])
DEBUG: d_valid shape before: torch.Size([2073600])
DEBUG: d_valid unsqueezed: torch.Size([2073600, 1])


OutOfMemoryError: CUDA out of memory. Tried to allocate 16018.07 GiB. GPU 4 has a total capacty of 47.41 GiB of which 46.82 GiB is free. Including non-PyTorch memory, this process has 594.00 MiB memory in use. Of the allocated memory 86.11 MiB is allocated by PyTorch, and 15.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

# %% [markdown]
# ----
# ## D4. 准备 GT surface 点云（coverage_mesh.obj）

# %%
gt_mesh_path = RAW_ROOT / "coverage_mesh.obj"
assert gt_mesh_path.exists(), f"coverage_mesh.obj not found: {gt_mesh_path}"

mesh_gt = o3d.io.read_triangle_mesh(str(gt_mesh_path))
mesh_gt.compute_vertex_normals()
print(mesh_gt)

# 从 mesh 采样点云
num_gt_samples = 200000
pcd_gt = mesh_gt.sample_points_uniformly(number_of_points=num_gt_samples)
gt_points = np.asarray(pcd_gt.points).astype(np.float32)
print("GT sampled points:", gt_points.shape)

gt_ply_path = STAGE_A_ROOT / "gt_surface_points.ply"
o3d.io.write_point_cloud(str(gt_ply_path), pcd_gt)
print("Saved GT sampled points to:", gt_ply_path)


In [ ]:

# %% [markdown]
# ----
# ## D5. 最近邻距离 & Chamfer 风格评估（NeRF vs GT）

# %%
from sklearn.neighbors import NearestNeighbors

def nn_stats(src_pts: np.ndarray, dst_pts: np.ndarray, k: int = 1):
    """
    对于 src 中每个点，找 dst 中最近邻距离。
    返回 distances (N,) 和一些统计值。
    """
    nbrs = NearestNeighbors(n_neighbors=k, algorithm="kd_tree").fit(dst_pts)
    dists, _ = nbrs.kneighbors(src_pts)
    dists = dists[:, 0]  # (N,)
    stats = {
        "mean": float(dists.mean()),
        "median": float(np.median(dists)),
        "95%": float(np.percentile(dists, 95)),
        "max": float(dists.max()),
    }
    return dists, stats

# 为了算得快一点，可选 subsample
max_nerf_eval = 200000
if nerf_points.shape[0] > max_nerf_eval:
    idx = np.random.choice(nerf_points.shape[0], max_nerf_eval, replace=False)
    nerf_eval = nerf_points[idx]
else:
    nerf_eval = nerf_points

max_gt_eval = 200000
if gt_points.shape[0] > max_gt_eval:
    idx = np.random.choice(gt_points.shape[0], max_gt_eval, replace=False)
    gt_eval = gt_points[idx]
else:
    gt_eval = gt_points

print("NeRF eval pts:", nerf_eval.shape)
print("GT   eval pts:", gt_eval.shape)

# NeRF -> GT
d_nerf2gt, stats_nerf2gt = nn_stats(nerf_eval, gt_eval)

# GT -> NeRF
d_gt2nerf, stats_gt2nerf = nn_stats(gt_eval, nerf_eval)

print("NeRF → GT stats (meters):")
for k, v in stats_nerf2gt.items():
    print(f"  {k:6s}: {v}")

print("\nGT → NeRF stats (meters):")
for k, v in stats_gt2nerf.items():
    print(f"  {k:6s}: {v}")

chamfer_like = stats_nerf2gt["mean"] + stats_gt2nerf["mean"]
print(f"\nChamfer-like distance (mean n2g + mean g2n): {chamfer_like}")


In [ ]:

# %% [markdown]
# ----
# ## D6. 画直方图 + 三维可视化（风格和 Stage C 保持一致）

# %%
# 直方图（单位改成毫米更直观）
plt.figure(figsize=(12, 4))
bins = 80

plt.hist(d_nerf2gt * 1000.0, bins=bins, alpha=0.7, label="NeRF → GT")
plt.hist(d_gt2nerf * 1000.0, bins=bins, alpha=0.7, label="GT → NeRF")
plt.xlabel("NN distance [mm]")
plt.ylabel("Count")
plt.title("NN Distance Distributions: NeRF vs GT")
plt.legend()
plt.tight_layout()
plt.show()

# %%
# 三维散点（随机抽一点点看形状）
N_vis = 60000
if nerf_points.shape[0] > N_vis:
    idx = np.random.choice(nerf_points.shape[0], N_vis, replace=False)
    nerf_vis = nerf_points[idx]
else:
    nerf_vis = nerf_points

if gt_points.shape[0] > N_vis:
    idx = np.random.choice(gt_points.shape[0], N_vis, replace=False)
    gt_vis = gt_points[idx]
else:
    gt_vis = gt_points

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection="3d")

ax.scatter(gt_vis[:, 0], gt_vis[:, 1], gt_vis[:, 2], s=0.5, alpha=0.4, label="GT")
ax.scatter(nerf_vis[:, 0], nerf_vis[:, 1], nerf_vis[:, 2], s=0.5, alpha=0.4, label="NeRF")

ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_zlabel("Z (m)")
ax.set_title("Point clouds: GT vs NeRF (subsampled)")
ax.legend()
ax.set_box_aspect([1, 1, 1])
plt.tight_layout()
plt.show()
